<a href="https://colab.research.google.com/github/swoo-nam/project_final_team1/blob/main/%EC%84%9C%EB%B9%84%EC%8A%A42_%EB%8D%B0%EC%9D%B4%ED%84%B0%EC%A0%95%EC%A0%9C4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import pandas as pd
import numpy as np
import torch
from tqdm.auto import tqdm
import random
import os
import re

def reset_seeds(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

DATA_PATH = "/content/drive/MyDrive/Final_Project/data/"
SEED = 42

device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cpu'

# 요기요 데이터 불러오기

In [56]:
yo_review =pd.read_csv(f"{DATA_PATH}yogiyo_reviews_all.csv")['고객리뷰']
yo_reply = pd.read_csv(f"{DATA_PATH}yogiyo_reviews_all.csv")['사장댓글']

train = pd.concat([yo_review, yo_reply], axis=1)
train.head()

,고객리뷰,사장댓글
0,사진보다 실물이 백배 맛있어요~~~~~~ 고기 양도 많아요~~~~~~ 배달시간도 딱...,"이렇게 저희 매장 찾아주셔서 감사드립니다!!\n\n맛 뿐만 아니라 양도, 서비스도 ..."
1,리뷰가 늦었네요.. 맛있게 잘 먹었습니다. 또 시킬게요!!,좋은 리뷰 덕분에 기분이 행복해지는 것 같아요😇\n\n고객님께서 맛있다고 말씀 주시...
2,잘 먹었습니다. 다만 주문에 있던 펩시콜라는 받지 못했습니다.,소중한 식사 시간에 저희 매장을 찾아 주셔서 감사드립니다. \n\n저희를 찾아 주신...
3,굿 말이 필요 없는 메뉴 였습니다,수 많은 매장들을 헤치고 와주셔서 감사합니다!!\n\n저희 매장의 메뉴를 제대로 즐...
4,맛있어요 양도많구좋음,수 많은 매장들을 헤치고 와주셔서 감사합니다😂\n\n메뉴 그 자체로 행복을 드리고 ...


In [57]:
train.shape

(18686, 2)

In [58]:
rename_cols = ['review', 'reply']
train.columns = rename_cols
train.head()

,review,reply
0,사진보다 실물이 백배 맛있어요~~~~~~ 고기 양도 많아요~~~~~~ 배달시간도 딱...,"이렇게 저희 매장 찾아주셔서 감사드립니다!!\n\n맛 뿐만 아니라 양도, 서비스도 ..."
1,리뷰가 늦었네요.. 맛있게 잘 먹었습니다. 또 시킬게요!!,좋은 리뷰 덕분에 기분이 행복해지는 것 같아요😇\n\n고객님께서 맛있다고 말씀 주시...
2,잘 먹었습니다. 다만 주문에 있던 펩시콜라는 받지 못했습니다.,소중한 식사 시간에 저희 매장을 찾아 주셔서 감사드립니다. \n\n저희를 찾아 주신...
3,굿 말이 필요 없는 메뉴 였습니다,수 많은 매장들을 헤치고 와주셔서 감사합니다!!\n\n저희 매장의 메뉴를 제대로 즐...
4,맛있어요 양도많구좋음,수 많은 매장들을 헤치고 와주셔서 감사합니다😂\n\n메뉴 그 자체로 행복을 드리고 ...


## 특수문자 제거

In [59]:
review_list = []
for review in train['review']:
    if isinstance(review, str):
        # pat = re.compile("[^a-zA-Z가-힣ㄱ-ㅎ0-9]")
        # pat = re.compile("[^a-zA-Z가-힣ㄱ-ㅎ0-9\(\)\{\}\[\]]")
        # pat = re.compile("[^a-zA-Z가-힣0-9\(\)\{\}\[\]]")
        # pat = re.compile("[^a-zA-Z가-힣0-9]")
        pat = re.compile("[^a-zA-Z가-힣0-9.,~?!]")
        result = pat.sub(" ", review)

        result = re.sub(r'\.{2,}', '.', result)
        result = re.sub(r'\,{2,}', ',', result)
        result = re.sub(r'\~{2,}', '~', result)
        result = re.sub(r'\!{2,}', '!', result)
        result = re.sub(r'\?{2,}', '?', result)
        result = re.sub(" +", " ", result)
        review_list.append(result)
    else:
        review_list.append("")

train['review'] = review_list
train.head()

,review,reply
0,사진보다 실물이 백배 맛있어요~ 고기 양도 많아요~ 배달시간도 딱 맞춰서 보내주셨어요~,"이렇게 저희 매장 찾아주셔서 감사드립니다!!\n\n맛 뿐만 아니라 양도, 서비스도 ..."
1,리뷰가 늦었네요. 맛있게 잘 먹었습니다. 또 시킬게요!,좋은 리뷰 덕분에 기분이 행복해지는 것 같아요😇\n\n고객님께서 맛있다고 말씀 주시...
2,잘 먹었습니다. 다만 주문에 있던 펩시콜라는 받지 못했습니다.,소중한 식사 시간에 저희 매장을 찾아 주셔서 감사드립니다. \n\n저희를 찾아 주신...
3,굿 말이 필요 없는 메뉴 였습니다,수 많은 매장들을 헤치고 와주셔서 감사합니다!!\n\n저희 매장의 메뉴를 제대로 즐...
4,맛있어요 양도많구좋음,수 많은 매장들을 헤치고 와주셔서 감사합니다😂\n\n메뉴 그 자체로 행복을 드리고 ...


In [60]:
reply_list = []
for reply in train['reply']:
    if isinstance(reply, str):
        # pat = re.compile("[^a-zA-Z가-힣ㄱ-ㅎ0-9]")
        # pat = re.compile("[^a-zA-Z가-힣ㄱ-ㅎ0-9\(\)\{\}\[\]]")
        # pat = re.compile("[^a-zA-Z가-힣0-9\(\)\{\}\[\]]")
        # pat = re.compile("[^a-zA-Z가-힣0-9]")
        pat = re.compile("[^a-zA-Z가-힣0-9.,~?!]")
        result = pat.sub(" ", reply)

        result = re.sub(r'\.{2,}', '.', result)
        result = re.sub(r'\,{2,}', ',', result)
        result = re.sub(r'\~{2,}', '~', result)
        result = re.sub(r'\!{2,}', '!', result)
        result = re.sub(r'\?{2,}', '?', result)
        result = re.sub(" +", " ", result)
        reply_list.append(result)
    else:
        reply_list.append("")

train['reply'] = reply_list
train.head()

,review,reply
0,사진보다 실물이 백배 맛있어요~ 고기 양도 많아요~ 배달시간도 딱 맞춰서 보내주셨어요~,"이렇게 저희 매장 찾아주셔서 감사드립니다! 맛 뿐만 아니라 양도, 서비스도 모두 최..."
1,리뷰가 늦었네요. 맛있게 잘 먹었습니다. 또 시킬게요!,좋은 리뷰 덕분에 기분이 행복해지는 것 같아요 고객님께서 맛있다고 말씀 주시니 피로...
2,잘 먹었습니다. 다만 주문에 있던 펩시콜라는 받지 못했습니다.,소중한 식사 시간에 저희 매장을 찾아 주셔서 감사드립니다. 저희를 찾아 주신 만큼 ...
3,굿 말이 필요 없는 메뉴 였습니다,수 많은 매장들을 헤치고 와주셔서 감사합니다! 저희 매장의 메뉴를 제대로 즐기신 거...
4,맛있어요 양도많구좋음,수 많은 매장들을 헤치고 와주셔서 감사합니다 메뉴 그 자체로 행복을 드리고 싶었는데...


- 지점명 제거

In [61]:
pattern = re.compile(r'\S+점')
train["reply"] = train["reply"].apply(lambda x: pattern.sub('', x)) #__점 제거

In [62]:
train['reply'][446]

'안녕하세요! 당신은 지금 치킨이 땡긴다  입니다 헉, 칭찬 너무 감사합니다 항상 한결같은 양과 맛으로 보답하겠습니다! 고객님께서 치킨이 땡기실 때마다 저희가 생각나도록 더욱 열심히 하겠습니다아 주문 정말 감사드리고 다음에 또 봐요 '

In [63]:
train['reply'][36]

'주문해주셔서 감사합니다 항상 생각하지만, 저희 매장을 찾아 주시고 맛있다고 말씀을 주시면 메뉴를 제공해드리는 저희 입장에서는 너무 보람을 느끼고 감사할 따름입니다 다만, 밥 양이 조금 부족했던 점 죄송합니다! 양이 부족하실 것 같다면 추가 부탁드리겠습니다! 소중한 시간 내어 저희 매장 찾아주셔서 감사드립니다! '

In [64]:
train['reply'][820]

' 안녕하세요 고돼지 입니다  그리고 따뜻한 리뷰 작성해주셔서 감사합니다. 위생과 청결을 우선시하며 항상 내 가족이 먹는다는 마음으로 푸짐하고 든든한 한끼 식사가 되실수있도록 노력하는 고기맛집이 되겠습니다. 송파구 고기1등 배달맛집 GO돼지 '

- __님 제거(배달기사님, 부모님 유지)

In [65]:
train['reply'] = train['reply'].str.replace(r'\w+\s* 님', '고객님', regex=True)

In [66]:
train['reply'][252]

'안녕하세요 배달이 늦어져 죄송합니다. 한파로 길이 얼어서 배달기사님들이 상황이 여유치가 않았다고 합니다. 주문해주셔서 감사합니다.'

In [67]:
train['reply'][18508]

'안녕하세요 귀한 식사에 보돌미역 을 찾아주셔서 정말 감사드립니다 부모님께서 맛있게 드셨다니 진심으로 감사의 말씀 올립니다. 큰 사랑에 더큰 정성으로 준비할께용 새해 복많이 받으시고 가족분 모두 항상 건강하시길 기원드립니다 '

In [68]:
train['reply'][3844]

'고객님, 안녕하세요 시간내어 리뷰작성해주셔서 정말 감사합니다 항상 고객님들 주문 하나하나에 정성을 담아 대접하겠습니다. 믿고 드실수있는 갈비민족 기억해주세용~! 감사합니다 찜 댓글 무료고기100g 추가해드리고있읍니다. 많이이용해주세요'

In [69]:
train['reply'][7374]

'안녕하세요 고객님, 맛있게 드시고 소중한 리뷰와  5개 감사합니다 두 번째 주문도 만족하셨다니 정말 뿌듯하고 기쁩니다~! Wow ! 자주 찾아주시는만큼 더욱 최선을 다하는 김포불닭발이 되겠습니다. 생각나실 때 또 찾아주세요~ '

In [70]:
train['reply'][14863]

' 이쁜 리뷰 감사합니다 고객님께서 저희 알촌을 찾아주셔서 너무 기쁩니다 항상 신선한 재료로 엄선하여 조리를 하고있습니다. 건강한 한끼 한그릇에 담아드리는 알촌이 되겠습니다 오늘도 찾아주셔서 너무 감사합니다 오늘 하루도 행복하고 좋은 하루 되세요 !'

## 가게명 키워드 컬럼화

In [73]:
ygy_store_lst = ["1988응답하라추억의옛날도시락-방이점", "워커스커피 로스터스", "떡깨비-가락점", "당치땡-헬리오시티점", "우시장국밥-석촌점",  "고돼지-송파점",  "마라의신마라탕&마라샹궈",  "춘리마라탕-송파점",  "손수경의육회담은연어-가락점",  "무국적식탁-송파점",  "청년치킨-삼전점",  "송파만다린",  "배달의쌀국수-송파점",  "닭강점-헬리오시티점",  "토핑폭탄김치찜&김치찌개-송파점",  "조형훈족발보쌈",  "여왕의브런치",  "모어댄버터",  "플러스82-송파점",  "더티베지",  "잭아저씨족발보쌈-본점",  "육회&연어-미아본점",  "갈비민족-본점",  "구구족-성신여대역점",  "피자보이시나-대학로점",  "왕빈자삼파전-미아본점",  "행복한찜닭-성북점",  "스시사소우",  "저팔계&중국&마라탕&덮밥&요리&전문점",  "명작파스타-성북점",  "올데이파스타-동대문구점",  "대한냉면-성북점",  "인생제육",  "고기듬뿍미트박스-성북점",  "피자스웨그-성북점",  "청춘식당묵은지김치찜",  "삽교원조두리곱창",  "뜸들이다-북서울꿈의숲점",  "더팔당매운오징어&닭발 김치찜",  "우리집반찬도시락-혜화막내딸지점",  "수피자",  "마라쿡",  "김포불닭발",  "헬로팬닭갈비&마약볶음밥",  "대찌 대파생고기김치찌개&정육왕국물두루치기",  "구래상회",  "승도리네곱도리탕",  "자성당",  "수락",  "나인곱창",  "어시장",  "돈까스회관",  "호랑이초밥",  "땡초곱창막창",  "피자이탈리",  "텐텐마라탕",  "과일에반하다",  "삼다수",  "병천토속순대",  "최고남제육명가",  "짱닭치킨",  "걸작떡볶이",  "직화삼겹직구삼",  "짜글이가돼었소",  "부대찌개자신있는집",  "토핑폭탄김치찜&김치찌개-부천점",  "1989마라탕-부천점",  "미친피자-부평점",  "천년닭강정-부천부평점",  "수상한삼겹살-부천점",  "수제죽전문점-본가진죽-본점",  "오늘은 분식",  "이끌림마라탕",  "탐나는피자-부평점",  "고기혁명-부천점",  "라화방마라탕-신중동점",  "전금례닭볶음탕-부천점",  "큰아들백순대",  "공주닭발-부천시청점",  "신자매김치찜&김치찌개",  "짱닭치킨-도촌점",  "이태리면가게-야탑점",  "폴트버거-판교점",  "랜돌프뉴욕페페로니-야탑점",  "홍싸롱-수제돈까스&파스타",  "쇼부덮밥",  "카산도-정자본점",  "부성초밥-미금본점",  "닭장수섭삼계탕&전기구이통닭",  "깨돌이김밥-미금점",  "울진죽변항",  "GTS버거-분당정자점",  "코브라독스-분당미금역점",  "마라하오-죽전점",  "땅끝수산",  "초밥대통령-용인수지점",  "올데이케밥&샐러드",  "야키토리 카마쿠라",  "비오키친-서현본점",  "부산오빠1인국밥-분당점",  "알촌",  "밈피자",  "고씨네카레",  "홍스족발",  "뜸들이다",  "감성커피",  "짚신스시",  "떡형",  "39도시락",  "피제이피자",  "황궁쟁반짜장",  "OK반점",  "모모타코야키&카페",  "꼬알라파이",  "진지덮밥",  "피자프라텔로",  "한식세끼1인김치찜&김치찌개",  "윤희횟집",  "시민보쌈족발&감자탕",  "카페봄봄",  "킹프레소빅와플",  "최가네한쌈",  "창타이누들",  "보돌미역",  "홍대개미"  ]

store_name = []
for store in ygy_store_lst:
    pat = re.split(r'[- &]', store)
    pat = ' '.join(pat)
    store_name.append(pat)

In [76]:
store_name

['1988응답하라추억의옛날도시락 방이점',
 '워커스커피 로스터스',
 '떡깨비 가락점',
 '당치땡 헬리오시티점',
 '우시장국밥 석촌점',
 '고돼지 송파점',
 '마라의신마라탕 마라샹궈',
 '춘리마라탕 송파점',
 '손수경의육회담은연어 가락점',
 '무국적식탁 송파점',
 '청년치킨 삼전점',
 '송파만다린',
 '배달의쌀국수 송파점',
 '닭강점 헬리오시티점',
 '토핑폭탄김치찜 김치찌개 송파점',
 '조형훈족발보쌈',
 '여왕의브런치',
 '모어댄버터',
 '플러스82 송파점',
 '더티베지',
 '잭아저씨족발보쌈 본점',
 '육회 연어 미아본점',
 '갈비민족 본점',
 '구구족 성신여대역점',
 '피자보이시나 대학로점',
 '왕빈자삼파전 미아본점',
 '행복한찜닭 성북점',
 '스시사소우',
 '저팔계 중국 마라탕 덮밥 요리 전문점',
 '명작파스타 성북점',
 '올데이파스타 동대문구점',
 '대한냉면 성북점',
 '인생제육',
 '고기듬뿍미트박스 성북점',
 '피자스웨그 성북점',
 '청춘식당묵은지김치찜',
 '삽교원조두리곱창',
 '뜸들이다 북서울꿈의숲점',
 '더팔당매운오징어 닭발 김치찜',
 '우리집반찬도시락 혜화막내딸지점',
 '수피자',
 '마라쿡',
 '김포불닭발',
 '헬로팬닭갈비 마약볶음밥',
 '대찌 대파생고기김치찌개 정육왕국물두루치기',
 '구래상회',
 '승도리네곱도리탕',
 '자성당',
 '수락',
 '나인곱창',
 '어시장',
 '돈까스회관',
 '호랑이초밥',
 '땡초곱창막창',
 '피자이탈리',
 '텐텐마라탕',
 '과일에반하다',
 '삼다수',
 '병천토속순대',
 '최고남제육명가',
 '짱닭치킨',
 '걸작떡볶이',
 '직화삼겹직구삼',
 '짜글이가돼었소',
 '부대찌개자신있는집',
 '토핑폭탄김치찜 김치찌개 부천점',
 '1989마라탕 부천점',
 '미친피자 부평점',
 '천년닭강정 부천부평점',
 '수상한삼겹살 부천점',
 '수제죽전문점 본가진죽 본점',
 '오늘은 분식',
 '이끌림마라탕',
 '탐나는피자 

In [75]:
short_name = []
for name in store_name:
    for length in [2, 3, 4]:
        cut = name[:length]
        short_name.append(cut)
    if name[0].isdigit(): # 숫자로 시작하면 텍스트가 나오는 부분까지 추출
        cut = name[:6]
        short_name.append(cut)

short_name = [name for name in short_name if not name.isdigit()] # 숫자로만 되어 있는 키워드 삭제
short_name = list(set(short_name))
print(len(short_name), short_name)

342 ['랜돌프뉴', '닭강', '행복한찜', '피제이피', '배달의', '승도리네', '공주닭', '부성초밥', '우리', '더티베지', '오늘은 ', '나인곱', '육회', '보돌미', '워커', '탐나', '야키토리', '삽교원조', '대찌 대', '돈까스회', '고기혁', '황궁', '우리집', '짱닭치', '삽교원', '호랑이', '전금례', '고씨', '삼다', '미친피자', '홍스', '윤희횟집', '창타이누', '걸작떡볶', '춘리마', '과일에', '진지덮', '떡깨비 ', '플러', '땅끝수산', '텐텐마라', '고씨네카', '잭아저씨', '시민보', '피자이탈', '천년', '명작', '토핑', '땅끝', '떡형', '짚신스', '천년닭강', '조형', '수피자', '승도', '꼬알라파', '춘리', '홍싸', '과일', '인생제육', '손수경', '토핑폭', '청년', '홍스족발', '수상한삼', 'OK', '홍대', '쇼부덮', '감성커피', '깨돌', '나인곱창', '코브라', '고돼지', '짚신스시', '마라하', '마라', '올데이케', '고기듬', '텐텐', '짱닭치킨', '모모', '저팔계 ', '비오키친', '킹프', '이끌', '홍싸롱 ', '39도시락', '수제죽', '피자프라', '피제이', '시민', '여왕의브', '명작파스', '미친', '자성', '피제', '육회 연', '피자프', '우시장국', 'OK반', '돈까스', '윤희', '플러스', '헬로팬닭', '최고남', '춘리마라', '송파', '부대찌', '피자스', '오늘은', '쇼부', '떡깨비', '모어', '당치땡 ', '최가네', '모모타코', '마라의신', '전금', '윤희횟', '청춘', '밈피', '구래상', '인생제', '나인', '더티베', '직화', '걸작떡', '꼬알', '코브라독', '우시', '잭아저', '토핑폭탄', '킹프레', '수제', '구래상회', '더팔', '폴트버거', '호랑', '부성초', '육회 ', '모어댄', '직화삼겹

In [92]:
# short_name 에서 가게명 이외 일상용어로 사용되는 키워드는 제거
drop_name = ['OK','감성','걸작','공주','과일','김포','대한','땡초','명작','미친','배달','병천','부대','부산','비오','삽교','손수','송파','쇼부','수상','수제','시민','오늘', '오늘은 ', '우리', '울진', '이끌', '이끌림', '인생', '직화', '진지', '천년', '청년', '청춘', '최고', '큰아', '큰아들', '탐나', '토핑', '행복', '행복한', '헬로', '홍대','육회','배달의','야키토리','호랑이','우리집','한식']
short_name = [col for col in short_name if col not in drop_name]
print(len(short_name), short_name)

292 ['랜돌프뉴', '닭강', '행복한찜', '피제이피', '승도리네', '공주닭', '부성초밥', '더티베지', '나인곱', '보돌미', '워커', '삽교원조', '대찌 대', '돈까스회', '고기혁', '황궁', '짱닭치', '삽교원', '전금례', '고씨', '삼다', '미친피자', '홍스', '윤희횟집', '창타이누', '걸작떡볶', '춘리마', '과일에', '진지덮', '떡깨비 ', '플러', '땅끝수산', '텐텐마라', '고씨네카', '잭아저씨', '시민보', '피자이탈', '땅끝', '떡형', '짚신스', '천년닭강', '조형', '수피자', '승도', '꼬알라파', '춘리', '홍싸', '인생제육', '손수경', '토핑폭', '홍스족발', '수상한삼', '쇼부덮', '감성커피', '깨돌', '나인곱창', '코브라', '고돼지', '짚신스시', '마라하', '마라', '올데이케', '고기듬', '텐텐', '짱닭치킨', '모모', '저팔계 ', '비오키친', '킹프', '홍싸롱 ', '39도시락', '수제죽', '피자프라', '피제이', '여왕의브', '명작파스', '자성', '피제', '육회 연', '피자프', '우시장국', 'OK반', '돈까스', '윤희', '플러스', '헬로팬닭', '최고남', '춘리마라', '부대찌', '피자스', '떡깨비', '모어', '당치땡 ', '최가네', '모모타코', '마라의신', '전금', '윤희횟', '밈피', '구래상', '인생제', '나인', '더티베', '걸작떡', '꼬알', '코브라독', '우시', '잭아저', '토핑폭탄', '킹프레', '구래상회', '더팔', '폴트버거', '호랑', '부성초', '육회 ', '모어댄', '직화삼겹', '홍스족', '1989마라', '신자', '갈비', '왕빈자', '야키토', '올데', '이태리면', '카산도 ', '고씨네', '폴트', '자성당', '스시', '카산도', '잭아', '땅끝수', '땡초곱창', '삼다수', '김포불', '라화방', '황궁쟁', '3

In [93]:
train['store'] = ""
for name in short_name:
    train[name] = train['reply'].apply(lambda x: name if name in x else '') # 우선 각 가게명 컬럼 생성(그렇게해야 덮어쓰기가 안됨)
    train['store'] = train.apply(lambda row: row['store'] + ',' + row[name] if row[name] else row['store'], axis=1) # 하나로 합치기(맨 앞에 콤마가 생김)

train['store'] = train['store'].apply(lambda x: x[1:] if x.startswith(',') else x) # 맨 앞에 콤마 제거
train = train.drop(columns=short_name) # 각 가게명 컬럼 삭제

train.head()

,review,reply,store
0,사진보다 실물이 백배 맛있어요~ 고기 양도 많아요~ 배달시간도 딱 맞춰서 보내주셨어요~,"이렇게 저희 매장 찾아주셔서 감사드립니다! 맛 뿐만 아니라 양도, 서비스도 모두 최...",
1,리뷰가 늦었네요. 맛있게 잘 먹었습니다. 또 시킬게요!,좋은 리뷰 덕분에 기분이 행복해지는 것 같아요 고객님께서 맛있다고 말씀 주시니 피로...,
2,잘 먹었습니다. 다만 주문에 있던 펩시콜라는 받지 못했습니다.,소중한 식사 시간에 저희 매장을 찾아 주셔서 감사드립니다. 저희를 찾아 주신 만큼 ...,
3,굿 말이 필요 없는 메뉴 였습니다,수 많은 매장들을 헤치고 와주셔서 감사합니다! 저희 매장의 메뉴를 제대로 즐기신 거...,
4,맛있어요 양도많구좋음,수 많은 매장들을 헤치고 와주셔서 감사합니다 메뉴 그 자체로 행복을 드리고 싶었는데...,


In [109]:
train[train['reply'].str.contains(short_name[74])].index

Int64Index([2785, 2787, 2793, 2796], dtype='int64')

In [110]:
train['store'][2785]

'여왕의브,여왕의,여왕'

In [111]:
train['reply'][2785]

'소중한 고객님의 의견 너무 감사합니다. 더욱 노력하는 여왕의브런치 되겠습니다. 항상 행복하세요! 또 뵙겠습니다!'

In [80]:
train['store'][10216]

'미친피자,피자,미친피'

In [81]:
train['reply'][10216]

'안녕하세요! 좋은 식사가 되신것 같아 너무나도 뿌듯한 마음입니다 앞으로도 맛있는 피자가 생각나실때 저희 미친피자 또 찾아주세요 감사합니다 '

## 음식 키워드 컬럼화

- 전국통합식품영양성분정보(음식)표준데이터.csv
- https://www.data.go.kr/tcs/dss/selectStdDataDetailView.do

In [245]:
food_list = pd.read_csv(f"{DATA_PATH}food.csv")
food_list

,대표식품명
0,케이크
1,케이크
2,케이크
3,케이크
4,케이크
...,...
8298,청포묵무침
8299,치커리겉절이
8300,파래무침
8301,짜장소스


In [246]:
food_list = food_list.drop_duplicates(ignore_index = True)["대표식품명"]
food_list

0          케이크
1          치즈빵
2         카스텔라
3          츄러스
4         치아바타
        ...   
753      햄채소볶음
754    무생채(무만)
755     참나물겉절이
756     치커리겉절이
757       짜장소스
Name: 대표식품명, Length: 758, dtype: object

In [256]:
split_pattern = r'\s[\(/\-]s*'

In [258]:
food_list = (food_list.str.split(split_pattern).str.get(0))
food_list

0         케이크
1         치즈빵
2        카스텔라
3         츄러스
4        치아바타
        ...  
753     햄채소볶음
754       무생채
755    참나물겉절이
756    치커리겉절이
757      짜장소스
Name: 대표식품명, Length: 758, dtype: object

In [261]:
food_list = [word for word in food_list if len(word) > 1]

In [263]:
train['food'] = train['reply'].apply(lambda x: next((food for food in food_list if food in x), " "))
train['food']

0         
1         
2         
3         
4         
        ..
18681     
18682     
18683     
18684     
18685     
Name: food, Length: 18686, dtype: object

In [264]:
train['food'][700]

'국밥'

In [265]:
train['reply'][700]

'고객님, 소중한 고객이이이이임! 리뷰 넘 잘 써주셨자나요 감사해요! 꾸벅 사진도 먹음직스럽게 잘 찍어주셨군여 맛있게 드셔주셔서 너어어어어무 기쁘고 이번 주문으로 저희 우시장 국밥 에 찐팬이 되어 보시는건 어떠세요~ 언제나 변함 없는 맛으로 기다리고 있을테니 꼭 다시 한번 찾아주셨으면 좋겠어용! 약속 저희 우시장 국밥은 48시간 푹 끓여 고아낸 육수와 당일 도축한 신선한 고기들을 직접 손질하고 삶아 정말 정성을 다해 조리하고 있어요. 믿고 주문해주셔서 감사드립니다. 다음 주문도 기대해주세요! 고객님들의 의견을 새겨 듣고 계속 연마하여 더 나은 맛과 좋은 서비스로 보답해드릴게요 소중한 시간 내어 리뷰 남겨주셔서 다시 한번 감사드립니다. 또 뵙는 날까지 기다릴게요~ '

## 맞춤법 수정

In [5]:
!pip install git+https://github.com/ssut/py-hanspell.git

  Cloning https://github.com/ssut/py-hanspell.git to /tmp/pip-req-build-g2vwta7f
  Running command git clone --filter=blob:none --quiet https://github.com/ssut/py-hanspell.git /tmp/pip-req-build-g2vwta7f
  Resolved https://github.com/ssut/py-hanspell.git to commit fdc6ca50c19f1c85971437a072d89d4e5ce024b8
  Preparing metadata (setup.py) ... done
  Created wheel for py-hanspell: filename=py_hanspell-1.1-py3-none-any.whl size=4813 sha256=fc39d19a6a581d54d0f32f44a28034bd095a88e6cc942f4a274ef9e870c75875
  Stored in directory: /tmp/pip-ephem-wheel-cache-uk0ur6jv/wheels/2e/43/cc/753c9e1d91affb9ea40e186cea5654fb9231deb454da6724e5
Successfully built py-hanspell


In [112]:
from hanspell import spell_checker

def correct_spell(text):
    if isinstance(text, str):  # 입력 데이터가 문자열인 경우에만 맞춤법 검사 수행
        corrected_text = spell_checker.check(text).checked
        return corrected_text
    else:
        return text

In [116]:
tmp = train['reply'][800]
tmp

'소중한 고객니이이이임! 맛있게 드시고 별 다섯개 리뷰 써주셔서 넘넘 감사드려요오! 다음번에도 기회가 된다면 맛있는 식사 올릴수있도록 열심히 준비하고 있겠습니다아 오늘도 행복한 하루 보내세요 .~ 저희 우시장 국밥은 48시간 푹 끓여 고아낸 육수와 당일 도축한 신선한 고기들을 직접 손질하고 삶아 정말 정성을 다해 조리하고 있어요. 믿고 주문해주셔서 감사드립니다. 다음 주문도 기대해주세요! 고객님들의 의견을 새겨 듣고 계속 연마하여 더 나은 맛과 좋은 서비스로 보답해드릴게요 소중한 시간 내어 리뷰 남겨주셔서 다시 한번 감사드립니다. 또 뵙는 날까지 기다릴게요~ '

In [117]:
correct_spell(tmp)

'소중한 고객님이 이이임! 맛있게 드시고 별 다섯 개 리뷰 써주셔서 넘넘 감사드려요 오! 다음번에도 기회가 된다면 맛있는 식사 올릴 수 있도록 열심히 준비하고 있겠습니다 오늘도 행복한 하루 보내세요 .~ 저희 우시장 국밥은 48시간 푹 끓여 고아 낸 육수와 당일 도축한 신선한 고기들을 직접 손질하고 삶아 정말 정성을 다해 조리하고 있어요. 믿고 주문해 주셔서 감사드립니다. 다음 주문도 기대해 주세요! 고객님들의 의견을 새겨듣고 계속 연마하여 더 나은 맛과 좋은 서비스로 보답해 드릴게요 소중한 시간 내어 리뷰 남겨주셔서 다시 한번 감사드립니다. 또 뵙는 날까지 기다릴게요~ '

In [123]:
corrected_reply = [] # 사장댓글에만 적용
for review in tqdm(train['reply']):
    tmp = correct_spell(review)
    corrected_reply.append(tmp)

  0%|          | 0/18686 [00:00<?, ?it/s]

In [124]:
train["corrected_reply"] = corrected_reply

In [125]:
train.to_csv(f"{DATA_PATH}요기요_가게_맞춤법수정.csv",index=False) # 저장

In [126]:
train

,review,reply,store,corrected_reply
0,사진보다 실물이 백배 맛있어요~ 고기 양도 많아요~ 배달시간도 딱 맞춰서 보내주셨어요~,"이렇게 저희 매장 찾아주셔서 감사드립니다! 맛 뿐만 아니라 양도, 서비스도 모두 최...",,"이렇게 저희 매장 찾아주셔서 감사드립니다! 맛뿐만 아니라 양도, 서비스도 모두 최상..."
1,리뷰가 늦었네요. 맛있게 잘 먹었습니다. 또 시킬게요!,좋은 리뷰 덕분에 기분이 행복해지는 것 같아요 고객님께서 맛있다고 말씀 주시니 피로...,,좋은 리뷰 덕분에 기분이 행복해지는 것 같아요 고객님께서 맛있다고 말씀 주시니 피로...
2,잘 먹었습니다. 다만 주문에 있던 펩시콜라는 받지 못했습니다.,소중한 식사 시간에 저희 매장을 찾아 주셔서 감사드립니다. 저희를 찾아 주신 만큼 ...,,소중한 식사 시간에 저희 매장을 찾아 주셔서 감사드립니다. 저희를 찾아 주신 만큼 ...
3,굿 말이 필요 없는 메뉴 였습니다,수 많은 매장들을 헤치고 와주셔서 감사합니다! 저희 매장의 메뉴를 제대로 즐기신 거...,,수많은 매장들을 헤치고 와주셔서 감사합니다! 저희 매장의 메뉴를 제대로 즐기신 거 ...
4,맛있어요 양도많구좋음,수 많은 매장들을 헤치고 와주셔서 감사합니다 메뉴 그 자체로 행복을 드리고 싶었는데...,,수많은 매장들을 헤치고 와주셔서 감사합니다 메뉴 그 자체로 행복을 드리고 싶었는데 ...
...,...,...,...,...
18681,맛있게 잘 먹었어요,감사합니다! 다음에도 이용해주세용,,감사합니다! 다음에도 이용해 주세요
18682,요청사항을 항상 잘 들어주셨는데 오늘은 못 모셨나봐요 그래도 잘 먹었습니다,죄송합니다 ! 다음에도 애용해주세용,,죄송합니다! 다음에도 애용해 주세요
18683,배달 빨라요 맛있게 잘 먹었습니당 !,감사합니다! 다음에도 이용해주세용,,감사합니다! 다음에도 이용해 주세요
18684,잘 먹었어요~,감사합니다! 다음에도 이용해주세용,,감사합니다! 다음에도 이용해 주세요


## 불용어 제거 -> Kiwi

In [127]:
!pip install kiwipiepy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.5/30.5 MB 25.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for kiwipiepy-model: filename=kiwipiepy_model-0.15.0-py3-none-any.whl size=30602628 sha256=22df7af9b3a62b1e4fe329cd70c8ac1f299403690058e537f2d3b5d4334e631c
  Stored in directory: /root/.cache/pip/wheels/f3/55/41/ca474338ece1bc4314b01445f64ff002d71e19df45575a16e2
Successfully built kiwipiepy-model


In [128]:
from kiwipiepy import Kiwi

# Kiwi 객체 생성
kiwi = Kiwi()

In [212]:
stopwords = [('이임','NNG'),('이','MM'),('용','XSN'),('요','MM'),('너어어어무', 'NNG'),('너어어어어무','NNG')]

In [205]:
test = kiwi.tokenize(train["corrected_reply"][:100])

In [163]:
for tokens in test:
    print(tokens)

[Token(form='이렇게', tag='MAG', start=0, len=3), Token(form='저희', tag='NP', start=4, len=2), Token(form='매장', tag='NNG', start=7, len=2), Token(form='찾', tag='VV', start=10, len=1), Token(form='어', tag='EC', start=11, len=1), Token(form='주', tag='VX', start=12, len=1), Token(form='시', tag='EP', start=13, len=1), Token(form='어서', tag='EC', start=13, len=2), Token(form='감사', tag='NNG', start=16, len=2), Token(form='드리', tag='VV', start=18, len=2), Token(form='ᆸ니다', tag='EF', start=19, len=3), Token(form='!', tag='SF', start=22, len=1), Token(form='맛', tag='NNG', start=24, len=1), Token(form='뿐', tag='JX', start=25, len=1), Token(form='만', tag='JX', start=26, len=1), Token(form='아니', tag='VCN', start=28, len=2), Token(form='라', tag='EC', start=30, len=1), Token(form='양도', tag='NNG', start=32, len=2), Token(form=',', tag='SP', start=34, len=1), Token(form='서비스', tag='NNG', start=36, len=3), Token(form='도', tag='JX', start=39, len=1), Token(form='모두', tag='MAG', start=41, len=2), Token(form='

In [207]:
sentence = train['corrected_reply'][806]

In [208]:
kiwi.tokenize(sentence)

[Token(form='고객', tag='NNG', start=0, len=2),
 Token(form='님', tag='XSN', start=2, len=1),
 Token(form='3', tag='SN', start=4, len=1),
 Token(form='일', tag='NNB', start=5, len=1),
 Token(form='연속', tag='NNG', start=7, len=2),
 Token(form='이나', tag='JX', start=9, len=2),
 Token(form='들', tag='VV', start=12, len=1),
 Token(form='시', tag='EP', start=13, len=1),
 Token(form='다니', tag='EF', start=14, len=2),
 Token(form='.', tag='SF', start=16, len=1),
 Token(form='정말', tag='MAG', start=18, len=2),
 Token(form='감사', tag='NNG', start=21, len=2),
 Token(form='하', tag='XSV', start=23, len=1),
 Token(form='ᆸ니다', tag='EF', start=23, len=3),
 Token(form='아', tag='IC', start=27, len=1),
 Token(form='아앙', tag='IC', start=28, len=2),
 Token(form='!', tag='SF', start=30, len=1),
 Token(form='감동', tag='NNG', start=32, len=2),
 Token(form='그만큼', tag='MAG', start=35, len=3),
 Token(form='맛있', tag='VA', start=39, len=2),
 Token(form='다는', tag='ETM', start=41, len=2),
 Token(form='뜻', tag='NNG', start=44,

In [209]:
tokens = kiwi.tokenize(sentence)

In [213]:
result = [ (t.form, t.tag) for t in tokens if t.tag != 'IC' and (t.form,t.tag) not in stopwords  ]

In [214]:
kiwi.join(result)

'고객님 3일 연속이나 드시다니. 정말 감사합니다! 감동 그만큼 맛있다는 뜻이겠죠~? 기쁘네요 역시 국밥은 질리지 않죠~진리입니다! 씩 정도면 찐으로 우시장 국밥 팬 맞군요! 행복 저희도 고객님 팬 하면 안 될까요~? 궁금 하루하루 다른 메뉴로 골라 시켜 드셔 보시는 거 어떠세요~? 굿 항상 찾아 주시니 매일 고객님만 기다릴 거 같아요 두근두근 매일 찾아 주시는 만큼 변함 없는 맛으로 기다리고 있을게요~약속 그날까지 항상 건강하시고 좋은 일만 가 아득하시길 바랄게요! 하투 저희 우시장 국밥은 48시간 푹 끓여 과 낸 육수와 당일 도축한 신선한 고기들을 직접 손질하고 삶아 정말 정성을 다해 조리하고 있어요. 믿고 주문해 주셔서 감사 드립니다. 다음 주문도 기대해 주세요! 고객님들의 의견을 새겨듣고 계속 연마해 더 나은 맛과 좋은 서비스로 보답해 드릴게요 소중한 시간 내 어 리뷰 남겨 주셔서 다시 한 번 감사 드립니다. 또 뵙는 날까지 기다릴게요~'

In [267]:
tokenized_replyContents = []
kiwi_replyContents = []

for reply in train['corrected_reply']:
    result = kiwi.tokenize(reply)
    filtered_tokens = [ (t.form, t.tag) for t in result if t.tag != 'IC' and (t.form,t.tag) not in stopwords and t.tag != 'Z_CODA']
    tokenized_replyContents.append(filtered_tokens)

    kiwi_replyContent = kiwi.join(filtered_tokens)
    kiwi_replyContents.append(kiwi_replyContent)

In [269]:
train["kiwi_replyContents"] = kiwi_replyContents
train

,review,reply,store,corrected_reply,대표식품명,food,kiwi_replyContents
0,사진보다 실물이 백배 맛있어요~ 고기 양도 많아요~ 배달시간도 딱 맞춰서 보내주셨어요~,"이렇게 저희 매장 찾아주셔서 감사드립니다! 맛 뿐만 아니라 양도, 서비스도 모두 최...",,"이렇게 저희 매장 찾아주셔서 감사드립니다! 맛뿐만 아니라 양도, 서비스도 모두 최상...",,,"이렇게 저희 매장 찾아 주셔서 감사 드립니다! 맛뿐만 아니라 양도, 서비스도 모두 ..."
1,리뷰가 늦었네요. 맛있게 잘 먹었습니다. 또 시킬게요!,좋은 리뷰 덕분에 기분이 행복해지는 것 같아요 고객님께서 맛있다고 말씀 주시니 피로...,,좋은 리뷰 덕분에 기분이 행복해지는 것 같아요 고객님께서 맛있다고 말씀 주시니 피로...,,,좋은 리뷰 덕분에 기분이 행복해지는 것 같아요 고객님께서 맛있다고 말씀 주시니 피로...
2,잘 먹었습니다. 다만 주문에 있던 펩시콜라는 받지 못했습니다.,소중한 식사 시간에 저희 매장을 찾아 주셔서 감사드립니다. 저희를 찾아 주신 만큼 ...,,소중한 식사 시간에 저희 매장을 찾아 주셔서 감사드립니다. 저희를 찾아 주신 만큼 ...,,,소중한 식사 시간에 저희 매장을 찾아 주셔서 감사 드립니다. 저희를 찾아 주신 만큼...
3,굿 말이 필요 없는 메뉴 였습니다,수 많은 매장들을 헤치고 와주셔서 감사합니다! 저희 매장의 메뉴를 제대로 즐기신 거...,,수많은 매장들을 헤치고 와주셔서 감사합니다! 저희 매장의 메뉴를 제대로 즐기신 거 ...,,,수많은 매장들을 헤치고 와 주셔서 감사합니다! 저희 매장의 메뉴를 제대로 즐기신 거...
4,맛있어요 양도많구좋음,수 많은 매장들을 헤치고 와주셔서 감사합니다 메뉴 그 자체로 행복을 드리고 싶었는데...,,수많은 매장들을 헤치고 와주셔서 감사합니다 메뉴 그 자체로 행복을 드리고 싶었는데 ...,,,수많은 매장들을 헤치고 와 주셔서 감사합니다 메뉴 그 자체로 행복을 드리고 싶었는데...
...,...,...,...,...,...,...,...
18681,맛있게 잘 먹었어요,감사합니다! 다음에도 이용해주세용,,감사합니다! 다음에도 이용해 주세요,,,감사합니다! 다음에도 이용해 주세요
18682,요청사항을 항상 잘 들어주셨는데 오늘은 못 모셨나봐요 그래도 잘 먹었습니다,죄송합니다 ! 다음에도 애용해주세용,,죄송합니다! 다음에도 애용해 주세요,,,죄송합니다! 다음에도 애용해 주세요
18683,배달 빨라요 맛있게 잘 먹었습니당 !,감사합니다! 다음에도 이용해주세용,,감사합니다! 다음에도 이용해 주세요,,,감사합니다! 다음에도 이용해 주세요
18684,잘 먹었어요~,감사합니다! 다음에도 이용해주세용,,감사합니다! 다음에도 이용해 주세요,,,감사합니다! 다음에도 이용해 주세요


# 오버샘플링

In [297]:
star = pd.read_csv(f"{DATA_PATH}yogiyo_reviews_all.csv")["별점"]
star

0        5
1        5
2        5
3        5
4        5
        ..
18681    5
18682    4
18683    5
18684    5
18685    5
Name: 별점, Length: 18686, dtype: int64

In [298]:
train["star"] = star
train

,review,reply,store,corrected_reply,대표식품명,food,kiwi_replyContents,star
0,사진보다 실물이 백배 맛있어요~ 고기 양도 많아요~ 배달시간도 딱 맞춰서 보내주셨어요~,"이렇게 저희 매장 찾아주셔서 감사드립니다! 맛 뿐만 아니라 양도, 서비스도 모두 최...",,"이렇게 저희 매장 찾아주셔서 감사드립니다! 맛뿐만 아니라 양도, 서비스도 모두 최상...",,,"이렇게 저희 매장 찾아 주셔서 감사 드립니다! 맛뿐만 아니라 양도, 서비스도 모두 ...",5
1,리뷰가 늦었네요. 맛있게 잘 먹었습니다. 또 시킬게요!,좋은 리뷰 덕분에 기분이 행복해지는 것 같아요 고객님께서 맛있다고 말씀 주시니 피로...,,좋은 리뷰 덕분에 기분이 행복해지는 것 같아요 고객님께서 맛있다고 말씀 주시니 피로...,,,좋은 리뷰 덕분에 기분이 행복해지는 것 같아요 고객님께서 맛있다고 말씀 주시니 피로...,5
2,잘 먹었습니다. 다만 주문에 있던 펩시콜라는 받지 못했습니다.,소중한 식사 시간에 저희 매장을 찾아 주셔서 감사드립니다. 저희를 찾아 주신 만큼 ...,,소중한 식사 시간에 저희 매장을 찾아 주셔서 감사드립니다. 저희를 찾아 주신 만큼 ...,,,소중한 식사 시간에 저희 매장을 찾아 주셔서 감사 드립니다. 저희를 찾아 주신 만큼...,5
3,굿 말이 필요 없는 메뉴 였습니다,수 많은 매장들을 헤치고 와주셔서 감사합니다! 저희 매장의 메뉴를 제대로 즐기신 거...,,수많은 매장들을 헤치고 와주셔서 감사합니다! 저희 매장의 메뉴를 제대로 즐기신 거 ...,,,수많은 매장들을 헤치고 와 주셔서 감사합니다! 저희 매장의 메뉴를 제대로 즐기신 거...,5
4,맛있어요 양도많구좋음,수 많은 매장들을 헤치고 와주셔서 감사합니다 메뉴 그 자체로 행복을 드리고 싶었는데...,,수많은 매장들을 헤치고 와주셔서 감사합니다 메뉴 그 자체로 행복을 드리고 싶었는데 ...,,,수많은 매장들을 헤치고 와 주셔서 감사합니다 메뉴 그 자체로 행복을 드리고 싶었는데...,5
...,...,...,...,...,...,...,...,...
18681,맛있게 잘 먹었어요,감사합니다! 다음에도 이용해주세용,,감사합니다! 다음에도 이용해 주세요,,,감사합니다! 다음에도 이용해 주세요,5
18682,요청사항을 항상 잘 들어주셨는데 오늘은 못 모셨나봐요 그래도 잘 먹었습니다,죄송합니다 ! 다음에도 애용해주세용,,죄송합니다! 다음에도 애용해 주세요,,,죄송합니다! 다음에도 애용해 주세요,4
18683,배달 빨라요 맛있게 잘 먹었습니당 !,감사합니다! 다음에도 이용해주세용,,감사합니다! 다음에도 이용해 주세요,,,감사합니다! 다음에도 이용해 주세요,5
18684,잘 먹었어요~,감사합니다! 다음에도 이용해주세용,,감사합니다! 다음에도 이용해 주세요,,,감사합니다! 다음에도 이용해 주세요,5


In [299]:
train_all = train[['review','kiwi_replyContents','star','store','food']]

In [300]:
from sklearn.utils import resample

# 오버샘플링을 수행할 별점 분포 계산
star_distribution = train_all['star'].value_counts().sort_index()

print(star_distribution)

1       92
2       64
3      179
4      560
5    17791
Name: star, dtype: int64


In [308]:
# 오버샘플링을 수행할 최대 샘플 수 설정
max_samples = 2000

# 오버샘플링 수행
oversampled_dfs = []

for star, count in star_distribution.items():
    subset = train_all[train_all['star'] == star]
    # 별점이 5가 아닌 경우에만 오버샘플링 수행
    if star != 5:
        resampled = resample(subset, replace=True, n_samples=max_samples, random_state=SEED)
        oversampled_dfs.append(resampled)
    else:
        oversampled_dfs.append(subset)

ValueError: ignored

In [302]:
# 오버샘플링된 데이터프레임 합치기
OV_train = pd.concat(oversampled_dfs).reset_index(drop=True)

In [305]:
# 오버샘플링 후 별점 분포 확인
ov_dis = OV_train['star'].value_counts().sort_index()
ov_dis

1     2000
2     2000
3     2000
4     2000
5    17791
Name: star, dtype: int64

In [309]:
OV_train[OV_train['star'] == 1]

,review,kiwi_replyContents,star,store,food
0,처음배달온 사진.음식을 위아래로 흔들어왔나보네요 씬도우가 도우만 씬이 아니고 모든 ...,안녕하세요 먼저 배달의 불편을 드려서 정말 죄송합니다 대행 기사님들께 다시 한번 부...,1,,
1,머리카락나옴. 위생신경좀 국수와 숙주 비율 1 1 맛 없음. 이맛도 저맛도 아닌맛 ...,안녕하세요 고객님 다시 한번 사과의 말씀 드리고 싶습니다 오늘 직원들과의 미팅을 통...,1,,
2,별로에요.,저희 가성비를 자랑하는 초밥입니다~찾아 주셔서 감사합니다~,1,초밥,초밥
3,별점 1개도 아까워요,받으신 피자에 문제가 있으셨다면 매장으로 바로 전화 주시면 신속하게 해결해 드리겠습니다,1,피자,피자
4,오늘도 맛있게 먹게 해주셔서 감사합니다. 그런데 지난번 누룽지 사건 이후로 사장님 ...,안녕하세요 고객님 저희 매장 다시 찾아 주셨는데 만족 드리지 못해 죄송합니다 고의적...,1,,
...,...,...,...,...,...
1995,리뷰 이벤트 한다지만 너무 아닌 거 같아서 적습니다. 배달이 20분 넘게 안오길래 ...,안녕하세요 1988 응답하라 추억의 옛날 도시락입니다. 주신 리뷰 잘 읽었습니다. ...,1,1988응답,
1996,비오는날이라 배달 늦는건 이해한다고 해도. 버팔로봉이 비닐에 나뒹굴어서 기분나빠서 ...,"고객님 주문 주시고 늦은 배달, 그리고 제대로 포장되지 않은 더 팔로 봉 리뷰 누락...",1,,
1997,가격대비 좀 심함,돈코쓰 세트가 제일 인기 많은 메뉴인데 많이 부족했나 봅니다 죄송합니다,1,,
1998,맛은 평범합니다. 영수증에 주소까지나와있는데 전화하셔서 제대로 안나와있다고하셔서 사...,기분 나쁘셨다니 죄송합니다. 기사님이 뭔가 착오가 있으셨나 봅니다. 제가 오늘 대책...,1,,


In [310]:
OV_train.to_csv(f"{DATA_PATH}0830.csv",index=False) # 저장